In [101]:
import sys
import spotipy
import spotipy.util as util

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, minmax_scale

import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'
SPOTIPY_CLIENT_SECRET='c02f97ddd119470d8e24b290722c7490'
SPOTIPY_CLIENT_ID='647ffb8c163b4882be0cafe4bf1352d5'
SCOPE='user-library-read'
USERNAME = '29n1g5zgbzcs413wmovhqdguz'

In [3]:
token = util.prompt_for_user_token(username=USERNAME,
                           scope=SCOPE,
                           client_id=SPOTIPY_CLIENT_ID,
                           client_secret=SPOTIPY_CLIENT_SECRET,
                           redirect_uri=SPOTIPY_REDIRECT_URI)

if not token:
    print("Can't get token for username", username)
    sys.exit(1)

sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=647ffb8c163b4882be0cafe4bf1352d5&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-library-read in your browser


Enter the URL you were redirected to: http://localhost:8888/callback?code=AQAwZp8_VTyMPrq5T3EcG0mx7fMM2mGQXErqOldj1WqETJkXkLPg5cWpiMxPOU9pjM8VY5LxNwyWoWC34_z3KYE0bLZRE3PmrU2R9tcH6X_6y4XuHQ4FFakP8zWc-lXUDTgVCGsMu9OZBkQF_TP2fD0VWlGTSp25KRXFTkYmgsOklImJOBOQz-fLsdXfxOPj2gLyD5QcZNVQd1PR7msl2MU4ZYjVGTc




In [177]:
grunge_features = pickle.load(open("datasets/grunge_features", "rb"))
rap_features = pickle.load(open("datasets/rap_features", "rb"))
soul_features = pickle.load(open("datasets/soul_features", "rb"))
techno_features = pickle.load(open("datasets/techno_features", "rb"))

If we define the importance of a feature as the measure of the variability of the data for that feature, we can use PCA to assign each feature an importance value.

In [192]:
def sort_features_importance(data):
    data_columns = data.columns
    data_norm = scale(data)
    data_norm = pd.DataFrame(data_norm, columns=data_columns)
    
    feature_n = data.shape[1]

    pca = PCA()
    pca.fit(data_norm)
    
    features_importances = {x: 0 for x in range(feature_n)}
    
    for comp_i in range(feature_n):
        comp = pca.components_[comp_i]
        importances = comp * pca.explained_variance_ratio_[comp_i]

        for feature_i, importance in enumerate(importances):
            if importance > features_importances[feature_i]:
                features_importances[feature_i] = importance
    feature_indices = sorted(features_importances, key=lambda x: features_importances[x], reverse=True)

    return list(data_columns[feature_indices])

In [193]:
print(sort_features_importance(techno_features))
print(sort_features_importance(soul_features))
print(sort_features_importance(grunge_features))
print(sort_features_importance(rap_features))

['tempo', 'key', 'acousticness', 'instrumentalness', 'danceability', 'liveness', 'duration_ms', 'speechiness', 'valence', 'time_signature', 'mode', 'loudness', 'energy']
['acousticness', 'duration_ms', 'speechiness', 'liveness', 'instrumentalness', 'tempo', 'mode', 'loudness', 'key', 'energy', 'time_signature', 'valence', 'danceability']
['acousticness', 'key', 'duration_ms', 'instrumentalness', 'danceability', 'speechiness', 'liveness', 'time_signature', 'loudness', 'valence', 'tempo', 'mode', 'energy']
['duration_ms', 'key', 'speechiness', 'instrumentalness', 'liveness', 'mode', 'valence', 'time_signature', 'acousticness', 'danceability', 'tempo', 'loudness', 'energy']
